In [1]:
import pandas as pd

In [6]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from statsmodels.stats.multitest import multipletests

# Load the Excel file
df = pd.read_excel("Proteomics PCA all quantified proteins 07162024.xlsx")

# Transpose and rename based on first row
df_transposed = df.transpose()
df_transposed = df_transposed.rename(columns=df_transposed.iloc[0]).drop(df_transposed.index[0])

# Assign simplified sample column names
expected_columns = ['130C_3/3', '131N_3/3', '131C_3/3', '132N_3/3',  # AD E3/E3
                    '132C_3/4', '133N_3/4', '133C_3/4', '134N_3/4',  # AD E3/E4
                    '126_3/3', '127N_3/3', '127C_3/3', '128N_3/3',   # NC E3/E3
                    '128C_3/4', '129N_3/4', '129C_3/4', '130N_3/4']  # NC E3/E4

assert len(df_transposed.columns) == len(expected_columns), "Column mismatch"
df_transposed.columns = expected_columns

# Remove proteins with all NA or constant values across all samples
df_transposed = df_transposed.dropna(how='all')
df_transposed = df_transposed[df_transposed.nunique(axis=1) > 1]

# Define sample groups for comparison
samples = expected_columns
conditions = ['AD_E3E3'] * 4 + ['AD_E3E4'] * 4 + ['NC_E3E3'] * 4 + ['NC_E3E4'] * 4

# Convert to long format for modeling
df_long = df_transposed[samples].copy()
df_long.columns = pd.MultiIndex.from_arrays([samples, conditions], names=["Sample", "Condition"])
df_long = df_long.T.stack().reset_index()
df_long.columns = ['Sample', 'Condition', 'ProteinID', 'Abundance']
df_long["Abundance"] = pd.to_numeric(df_long["Abundance"], errors="coerce")

# Compare each group to NC_E3E3
comparisons = ['AD_E3E3', 'AD_E3E4', 'NC_E3E4']
all_results = []

for comp in comparisons:
    results = []
    df_sub = df_long[df_long["Condition"].isin([comp, "NC_E3E3"])]

    for protein in df_sub["ProteinID"].unique():
        data = df_sub[df_sub["ProteinID"] == protein]

        # Drop missing values
        data = data.dropna(subset=["Abundance"])

        # Skip if only one group or constant values
        if data["Condition"].nunique() < 2 or data["Abundance"].nunique() < 2:
            coef, pval = np.nan, np.nan
        else:
            try:
                data["Condition"] = pd.Categorical(data["Condition"], categories=["NC_E3E3", comp])
                model = smf.ols("Abundance ~ C(Condition)", data=data).fit()

                coef = model.params.get(f"C(Condition)[T.{comp}]", np.nan)
                pval = model.pvalues.get(f"C(Condition)[T.{comp}]", np.nan)
            except:
                coef, pval = np.nan, np.nan

        results.append({
            "ProteinID": protein,
            f"log2FC_{comp}_vs_NC_E3E3": coef,
            f"pval_{comp}_vs_NC_E3E3": pval
        })

    res_df = pd.DataFrame(results).set_index("ProteinID")
    res_df[f"padj_{comp}_vs_NC_E3E3"] = multipletests(res_df[f"pval_{comp}_vs_NC_E3E3"], method="fdr_bh")[1]
    all_results.append(res_df)

# Merge all comparisons
final_df = pd.concat(all_results, axis=1)
final_df.reset_index(inplace=True)
final_df

,ProteinID,log2FC_AD_E3E3_vs_NC_E3E3,pval_AD_E3E3_vs_NC_E3E3,padj_AD_E3E3_vs_NC_E3E3,log2FC_AD_E3E4_vs_NC_E3E3,pval_AD_E3E4_vs_NC_E3E3,padj_AD_E3E4_vs_NC_E3E3,log2FC_NC_E3E4_vs_NC_E3E3,pval_NC_E3E4_vs_NC_E3E3,padj_NC_E3E4_vs_NC_E3E3
0,E9PQD6,-42.625,0.519684,0.953727,-59.075,0.364479,0.587011,-60.900,0.345728,0.997706
1,O15067,-7.850,0.035928,0.784226,-10.200,0.012137,0.196010,-5.725,0.181299,0.997706
2,Q96HS1,-3.450,0.452352,0.949731,-8.500,0.031880,0.246882,-0.825,0.817766,0.998663
3,P49641-1,1.650,0.555517,0.954914,-3.750,0.254892,0.507228,-0.125,0.965296,0.998663
4,Q86U86,-0.375,0.888857,0.989883,-3.025,0.413521,0.623850,1.650,0.496937,0.997706
...,...,...,...,...,...,...,...,...,...,...
3171,P60709,-263.950,0.853803,0.983629,1301.950,0.500818,0.696409,-810.825,0.569392,0.997706
3172,P14136,6102.450,0.005983,0.648929,5229.625,0.030238,0.245676,4205.500,0.185121,0.997706
3173,Q13885,-910.850,0.386162,0.938680,-1577.900,0.018952,0.224741,387.025,0.457183,0.997706
3174,Q13813-2,715.350,0.696329,0.971247,1492.575,0.352461,0.581213,653.650,0.682447,0.997706


In [7]:
pip install gprofiler-official


In [9]:
import pandas as pd
from gprofiler import GProfiler

# Assume 'final_df' is your existing DataFrame from the screenshot

# 1. Extract the list of unique protein IDs to convert
protein_ids = final_df['ProteinID'].unique().tolist()

# 2. Initialize the g:Profiler client
gp = GProfiler(return_dataframe=True)

# 3. Perform the ID conversion
# NOTE: The organism is set to 'hsapiens' (human).
# Change this if your data is from a different species (e.g., 'mmusculus' for mouse).
conversion_results = gp.convert(organism='hsapiens',
                                query=protein_ids,
                                target_namespace='HGNC') # HGNC is the standard for human gene symbols

# 4. Create a clean mapping DataFrame from the results
if not conversion_results.empty:
    # We only need the original ID ('incoming') and the new gene symbol ('converted')
    mapping_df = conversion_results[['incoming', 'converted']].rename(columns={
        'incoming': 'ProteinID',
        'converted': 'GeneSymbol'
    })

    # 5. Merge the new GeneSymbol column into your original DataFrame
    # A 'left' merge ensures all your original rows are kept
    final_df = pd.merge(final_df, mapping_df, on='ProteinID', how='left')

# Display the first few rows of the updated DataFrame
final_df

,ProteinID,log2FC_AD_E3E3_vs_NC_E3E3,pval_AD_E3E3_vs_NC_E3E3,padj_AD_E3E3_vs_NC_E3E3,log2FC_AD_E3E4_vs_NC_E3E3,pval_AD_E3E4_vs_NC_E3E3,padj_AD_E3E4_vs_NC_E3E3,log2FC_NC_E3E4_vs_NC_E3E3,pval_NC_E3E4_vs_NC_E3E3,padj_NC_E3E4_vs_NC_E3E3,GeneSymbol_x,GeneSymbol_y
0,E9PQD6,-42.625,0.519684,0.953727,-59.075,0.364479,0.587011,-60.900,0.345728,0.997706,None,None
1,O15067,-7.850,0.035928,0.784226,-10.200,0.012137,0.196010,-5.725,0.181299,0.997706,PFAS,PFAS
2,Q96HS1,-3.450,0.452352,0.949731,-8.500,0.031880,0.246882,-0.825,0.817766,0.998663,PGAM5,PGAM5
3,P49641-1,1.650,0.555517,0.954914,-3.750,0.254892,0.507228,-0.125,0.965296,0.998663,None,None
4,Q86U86,-0.375,0.888857,0.989883,-3.025,0.413521,0.623850,1.650,0.496937,0.997706,PBRM1,PBRM1
...,...,...,...,...,...,...,...,...,...,...,...,...
3870,P14136,6102.450,0.005983,0.648929,5229.625,0.030238,0.245676,4205.500,0.185121,0.997706,RPL23AP1,HLA-F-AS1
3871,P14136,6102.450,0.005983,0.648929,5229.625,0.030238,0.245676,4205.500,0.185121,0.997706,RPL23AP1,RPL23AP1
3872,Q13885,-910.850,0.386162,0.938680,-1577.900,0.018952,0.224741,387.025,0.457183,0.997706,TUBB2A,TUBB2A
3873,Q13813-2,715.350,0.696329,0.971247,1492.575,0.352461,0.581213,653.650,0.682447,0.997706,SPTAN1,SPTAN1


In [18]:
final_df = final_df.drop_duplicates()


In [20]:
final_df

,ProteinID,log2FC_AD_E3E3_vs_NC_E3E3,pval_AD_E3E3_vs_NC_E3E3,padj_AD_E3E3_vs_NC_E3E3,log2FC_AD_E3E4_vs_NC_E3E3,pval_AD_E3E4_vs_NC_E3E3,padj_AD_E3E4_vs_NC_E3E3,log2FC_NC_E3E4_vs_NC_E3E3,pval_NC_E3E4_vs_NC_E3E3,padj_NC_E3E4_vs_NC_E3E3,GeneSymbol_x,GeneSymbol_y
0,E9PQD6,-42.625,0.519684,0.953727,-59.075,0.364479,0.587011,-60.900,0.345728,0.997706,None,None
1,O15067,-7.850,0.035928,0.784226,-10.200,0.012137,0.196010,-5.725,0.181299,0.997706,PFAS,PFAS
2,Q96HS1,-3.450,0.452352,0.949731,-8.500,0.031880,0.246882,-0.825,0.817766,0.998663,PGAM5,PGAM5
3,P49641-1,1.650,0.555517,0.954914,-3.750,0.254892,0.507228,-0.125,0.965296,0.998663,None,None
4,Q86U86,-0.375,0.888857,0.989883,-3.025,0.413521,0.623850,1.650,0.496937,0.997706,PBRM1,PBRM1
...,...,...,...,...,...,...,...,...,...,...,...,...
3870,P14136,6102.450,0.005983,0.648929,5229.625,0.030238,0.245676,4205.500,0.185121,0.997706,RPL23AP1,HLA-F-AS1
3871,P14136,6102.450,0.005983,0.648929,5229.625,0.030238,0.245676,4205.500,0.185121,0.997706,RPL23AP1,RPL23AP1
3872,Q13885,-910.850,0.386162,0.938680,-1577.900,0.018952,0.224741,387.025,0.457183,0.997706,TUBB2A,TUBB2A
3873,Q13813-2,715.350,0.696329,0.971247,1492.575,0.352461,0.581213,653.650,0.682447,0.997706,SPTAN1,SPTAN1


In [19]:
final_df.to_csv("Proteomics_AD.csv")

In [11]:
final_df.columns

Index(['ProteinID', 'log2FC_AD_E3E3_vs_NC_E3E3', 'pval_AD_E3E3_vs_NC_E3E3',
       'padj_AD_E3E3_vs_NC_E3E3', 'log2FC_AD_E3E4_vs_NC_E3E3',
       'pval_AD_E3E4_vs_NC_E3E3', 'padj_AD_E3E4_vs_NC_E3E3',
       'log2FC_NC_E3E4_vs_NC_E3E3', 'pval_NC_E3E4_vs_NC_E3E3',
       'padj_NC_E3E4_vs_NC_E3E3', 'GeneSymbol_x', 'GeneSymbol_y'],
      dtype='object')